<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Preprocess Data
</div>

In [ ]:
import pandas as pd

### Read raw data that we have collected

In [ ]:
raw_df = pd.read_csv("../data/raw/raw_data.csv")
raw_df.head()

-   Look like the `Note` column is all NaN, let check it:
    -   How many kind of data in the column?
    -   The ratio between data with no values and these other?

In [ ]:
# check the variety of value
print("Unique values in the column:",raw_df['Note'].unique())

# check the ratio between loss and other data
raw_df['Note'].count()

-   Look like it's contain some useful information, so we keep it

### Meaning of each columns and rows

// Explain here

In [ ]:
#TODO: Use code to explain if need

### Clear duplicates columns:
-   There is `Year Code` and `Year` columns, they have the same values, so we drop the `Year Code`.

In [ ]:
raw_df = raw_df.drop(columns=['Year Code'])
raw_df.head()

### Check for duplicate rows

In [ ]:
index = raw_df.index
detectDupSeries = index.duplicated(keep='first')
num_duplicated_rows = detectDupSeries.sum()

In [ ]:
if num_duplicated_rows == 0:
    print(f"Your raw data have no duplicated line.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Your raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data.!")

### Check for the distribution of each value associated with an item.

In [ ]:
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

In [ ]:
# make a DataFrame to store each item info about how their data distribute
data_distribute_df = pd.DataFrame()

for item in raw_df['Item'].unique():
    item_df = raw_df[raw_df['Item'] == item] # get rows that have the same item

    num_col_info_df = item_df.select_dtypes(exclude='object')
    num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

    num_col_info_df = num_col_info_df.transpose()

    num_col_info_df['Item'] = item
    
    data_distribute_df = pd.concat([data_distribute_df, num_col_info_df])

In [ ]:
data_distribute_df

### Determine the features that have a large number of missing values

In [ ]:
percent_missing = raw_df.isnull().sum() * 100 / len(raw_df)
missing_value_df = pd.DataFrame({'column_name': raw_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

-   We gonna choose >= 75% is missing a lot of values, so we have to drop the `Note` column (~96.364%).

In [ ]:
raw_df = raw_df.drop(columns='Note')
raw_df

### Check data type of each columns

In [ ]:
raw_df.info()

`Area Code`, `Element Code`, `Item Code`, `Year` have numeric type. However, their magnitude does not have a significance. And, `Year` actually represents a period instead of a number, `Area Code`, `Element Code`, `Item Code` is just a category. Thus, they can be convert to categorical type.

In [ ]:
for col in raw_df.drop(columns= 'Value').columns:
    raw_df[col] = raw_df[col].astype(str)
raw_df.info()

### Observation about the distribution of data
-   

### Save processed data to a new csv file

In [ ]:
raw_df.to_csv("../data/processed/processed_data.csv", index= False)